In [1]:
# Install dependencis in your terminal
# pip install mediapipe opencv-python


In [2]:
# Import all needed packages
import mediapipe as mp
import cv2
import numpy as np
import os
import uuid
import math
from interface import *

In [3]:
# Mediapipe confg variables
mp_drawing = mp.solutions.drawing_utils # Module for rendering the landmarks on video/image
mp_hands = mp.solutions.hands # Module for work with hands - detection and tracking 

In [6]:
# Active WebCam
from tokenize import Name


cap = cv2.VideoCapture(0)
img_height = cap.get(4)
img_width = cap.get(3)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence= 0.5) as hands:
    while cap.isOpened():
        _, frame = cap.read() # Catches the frame from video

        # Hands detections
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Change frame's colors into RGB scheme
        # image.flags.writeable = False # Set flag to false
        results = hands.process(image)  # Detections
        # image.flags.writeable = True # Set flagto true
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Rechange colors
        
        # Draw detections on frame
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS) # Draw detected points on image
                hand.landmark[8].x = denormalization(hand, 8, img_height, img_width)[0] # Change x coordinate with denormalized value
                hand.landmark[8].y = denormalization(hand, 8, img_height, img_width)[1] # Change y coordinate with denormalized value

            try:                
                coord_list = add_to_list(coord_list, hand) # Add coordinates (x,y) to numpy list                         
            except NameError:
                coord_list = np.array([[hand.landmark[8].x, hand.landmark[8].y]]) # If coord_list not exist, create variable with (x,y) coords)
                first_item = coord_list[0] # Remember first coords
                
            if coord_list.shape[0] == 10: # In case there is 10 pairs (x,y) - calculate the mean value of x, and y coordinates
                coord_mean = np.mean(coord_list, axis=0)
                # print(f"Mean value: {coord_mean}")

            # If the mean not vary more than 1 - save point
            try:
                if abs(coord_mean[0] - coord_list[coord_list.shape[0]-1][0]) <= 1 and abs(coord_mean[1] - coord_list[coord_list.shape[0]-1][1]) <= 1:
                    start_point = np.array([coord_mean[0], coord_mean[1]])
                    
                    # print(f"Start point: {start_point}")
            except NameError:
                coord_mean = []
            except IndexError:
                pass

            try:                
                cv2.line(image, (int(start_point[0]), int(start_point[1])), (int(hand.landmark[8].x), int(hand.landmark[8].y)), (255, 0, 0), 3)
            except NameError:
                pass
                # cv2.line(image, (int(hand.landmark[8].x), int(hand.landmark[8].y)), (int(hand.landmark[8].x), int(hand.landmark[8].y)), (255, 0, 0), 3)

        cv2.imshow('Hand Tracking', cv2.flip(image,1))  # Display frame; Flip display image as mirrored

        # Get last 8 bits from pressed key.
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release() # Release WebCam    
    cv2.destroyAllWindows()  
    print(coord_list)  
    del coord_list 



[[190.  82.]
 [188.  84.]
 [186.  84.]
 [187.  85.]
 [187.  88.]
 [189.  87.]
 [195.  97.]
 [192.  98.]
 [187. 114.]
 [181. 115.]]
